# 2. Minio as an S3 service (practical)

***

**ELMI 2021 NGFF Workshop**, 25 June 2021

***


## Summary


* 2.1 Minio Client (mc)
* 2.2 Connections
* 2.3 Adding your own
* 2.4 Your first download

## 2.1 Minio client

There are a number of different types of cloud storage and there are a number of tools that you can use to access your cloud storage, but here we're going to focus on a single one `mc`.

`mc` is provided by the minio project and is described as "a modern alternative to UNIX commands like ls, cat, cp, mirror, diff, find etc." The quickstart guide can be found under https://docs.minio.io/docs/minio-client-quickstart-guide.html For our purposes we'll focus on how to use it to upload and manage data in S3.

In [5]:
!mc

NAME:
  mc - MinIO Client for cloud storage and filesystems.

USAGE:
  mc [FLAGS] COMMAND [COMMAND FLAGS | -h] [ARGUMENTS...]

COMMANDS:
  alias      set, remove and list aliases in configuration file
  ls         list buckets and objects
  mb         make a bucket
  rb         remove a bucket
  cp         copy objects
  mirror     synchronize object(s) to a remote site
  cat        display object contents
  head       display first 'n' lines of an object
  pipe       stream STDIN to an object
  share      generate URL for temporary access to an object
  find       search for objects
  sql        run sql queries on objects
  stat       show object metadata
  mv         move objects
  tree       list buckets and objects in a tree format
  du         summarize disk usage recursively
  retention  set retention for object(s)
  legalhold  manage legal hold for object(s)
  diff       list differences in object name, size, and date between two buckets
  rm         remove objects
  version    

## 2.2 Connections

The minio project provides a safe space for you to learn about S3: https://play.minio.io:9000/minio/ Here we've used the `mc` command to find the access information:

 * **"AccessKey"** is basically a user name.
 * **"SecretKey"** is basically a password. 
 * The URL is our **"endpoint"**, which differentiates it from the S3 servers provided by Amazon.

You can log in to the webpage and explore what the many other users have upload at https://play.minio.io:9000/minio/

The other two important concepts are:
 * **"buckets"** which is roughly like a shared namespace with permissions
 * and **"keys"** which will get to in a second.

In [8]:
!mc config host list play

play
  URL       : https://play.min.io
  AccessKey : Q3AM3UQ867SPQQA43P2F
  SecretKey : zuf+tfteSlswRu7BJ86wekitnifILbZam1KYY3TG
  API       : S3v4
  Path      : auto



## 2.3 Adding your own

But EMBL has kindly provided us a bucket for this session which we need to connect to. Please enter the password you were sent below:

In [1]:
# Python block for capturing a password
import getpass
s3_pass = getpass.getpass()

········


In [2]:
!mc config host add elmi2021 https://idr-ftp.openmicroscopy.org elmi2021 $s3_pass

Added `elmi2021` successfully.


In [4]:
!mc ls elmi2021/idr-upload/elmi2021/

[2021-06-21 15:12:13 CEST]    84B README.txt
[2021-06-22 17:00:25 CEST]     0B idr0023/


In [5]:
!mc ls elmi2021/idr-upload/elmi2021/idr0023/

[2021-06-22 17:00:38 CEST]     0B idr0023/


In [17]:
!mc ls gbi/bioim/idr0062-zarrs/

[2021-01-29 12:59:59 CET]     0B 6001237.zarr/
[2021-01-29 12:59:59 CET]     0B 6001238.zarr/
[2021-01-29 12:59:59 CET]     0B 6001239.zarr/
[2021-01-29 12:59:59 CET]     0B 6001240.zarr/
[2021-01-29 12:59:59 CET]     0B 6001241.zarr/
[2021-01-29 12:59:59 CET]     0B 6001242.zarr/
[2021-01-29 12:59:59 CET]     0B 6001243.zarr/
[2021-01-29 12:59:59 CET]     0B 6001244.zarr/
[2021-01-29 12:59:59 CET]     0B 6001245.zarr/
[2021-01-29 12:59:59 CET]     0B 6001246.zarr/
[2021-01-29 12:59:59 CET]     0B 6001247.zarr/
[2021-01-29 12:59:59 CET]     0B 6001248.zarr/
[2021-01-29 12:59:59 CET]     0B 6001249.zarr/
[2021-01-29 12:59:59 CET]     0B 6001250.zarr/
[2021-01-29 12:59:59 CET]     0B 6001251.zarr/
[2021-01-29 12:59:59 CET]     0B 6001252.zarr/
[2021-01-29 12:59:59 CET]     0B 6001253.zarr/
[2021-01-29 12:59:59 CET]     0B 6001254.zarr/
[2021-01-29 12:59:59 CET]     0B 6001255.zarr/
[2021-01-29 12:59:59 CET]     0B 6001256.zarr/
[2021-01-29 12:59:59 CET]     0B 6001257.zarr/
[2021-01-29 1

In [18]:
!mc ls gbi/bioim/idr0023/

[2021-01-28 10:22:33 CET]    44B README.txt
[2021-01-28 10:21:53 CET] 2.0MiB trans_norm.tif
[2021-01-29 13:00:27 CET]     0B trans_norm.ome.zarr/


## 2.3.4 Your first download

Now you can see list the contents of the bucket and see that it's there. Note that since other people are working on the same bucket, it may look slightly different each time you run this command.

In [22]:
!mc cp gbi/bioim/idr0023/trans_norm.tif julia.tif

..._norm.tif:  2.04 MiB / 2.04 MiB  ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓  236.18 KiB/s 8s

In [20]:
!ls -ltrah /tmp/trans_norm.tif

-rw-r--r--  1 jamoore  wheel   2.0M Jan 29 13:00 /tmp/trans_norm.tif


### License
Copyright (C) 2021 University of Dundee. All Rights Reserved.
This program is free software; you can redistribute it and/or modify it
under the terms of the GNU General Public License as published by the
Free Software Foundation; either version 2 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful, but
WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY
or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for
more details. You should have received a copy of the GNU General
Public License along with this program; if not, write to the
Free Software Foundation,
Inc., 51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.